In [10]:
import random


class Sudoku:
    def __init__(self, init_values: list[int]) -> None:
        if (len(init_values) == 81):
            self.init_values = init_values
        else:
            raise ValueError("List's length must be 81.")
        self.rows = self._get_rows(init_values)
        
    def _get_rows(self, values: list[int]) -> dict:
        '''Return a dictionary of the values in each row'''
        rows = {}
        for i in range(9):
            rows[i] = values[i * 9: (i+1) *9]
        return rows
    
    def _get_columns(self, rows: dict) -> dict:
        '''Return a dictionary of the values in each column'''
        columns = {}
        for i in range(9):
            column = []
            for j in range(9):
                column.append(rows[j][i])
            columns[i] = column
        return columns
            
    def _get_grids(self, rows: dict) -> dict:
        '''Return a dictionary of the values in each grid'''
        grids = {}
        for j in range(3):
            for i in range(3):
                grids[3*j+i] = rows[3*j][i*3:(i*3)+3] + rows[3*j+1][i*3:(i*3)+3] + rows[3*j+2][i*3:(i*3)+3]
        return grids

    def display_board(self, rows: dict) -> None:
        for i in range(9):
            if i % 3 == 0 and i != 0:
                print("- - - - - - - - - - - ")
            for j in range(9):
                if j % 3 == 0 and j != 0:
                    print("| ", end="")
                if rows[i][j] == 0:
                    print('* ', end="")
                if rows[i][j] != 0:
                    print(rows[i][j], end=" ")
            print()

    def get_individual(self) -> dict:
        '''Get a individual's dicitonary of missing values in each row'''
        individual = {}
        for j, value in enumerate(self.rows.values()):
            missing_value = []
            for i in range(1, len(value) + 1):
                if i not in value:
                    missing_value.append(i)
            random.shuffle(missing_value)
            individual[j] = missing_value
        return individual

    def _get_individual_rows(self, individual: dict) -> dict:
        '''Return a dictionary of rows filled with the missing values'''
        rows = {}
        for i in range(9):
            row = []
            iterator = iter(individual[i])
            for j in range(9):
                if self.rows[i][j] == 0:
                    row.append(next(iterator))
                else:
                    row.append(self.rows[i][j])
            rows[i] = row
        return rows

    def fitness_function(self, individual: dict) -> int:
        '''Return a fitness function for given individual 
        and it is counted by sum up the number of unique digits in each column and grid'''
        rows = self._get_individual_rows(individual)
        columns = self._get_columns(rows)
        grids = self._get_grids(rows)
        cost = 0
        for i in range(9):
            col_set = set(columns[i])
            cost += len(col_set)
            grid_set = set(grids[i])
            cost += len(grid_set)
        return cost
    
    def get_merged_individual(self):
        '''Returns an idividual merged from a dict'''
        individual = self.get_individual()
        ind_long = []
        for values in individual.values():
            ind_long += values
        return ind_long
    
    def get_population(self, quantity=100):
        '''Returns a population of individuals in a list'''
        return [self.get_individual() for x in range(quantity)]


In [11]:
import random
import copy
import statistics

class GeneticAlgorithm:

    def __init__(self) -> None:
        self.quantity_of_population = 20000
        
        self.mutation_possibility = 0.01
        self.crosing_possibility = 0.7
        self.max_iteration = 25
        self.hyperparameters = {
            'quantity_of_population': self.quantity_of_population,
            "mutation_possibility": self.mutation_possibility,
            'crossing_possibility': self.crosing_possibility,
            'max_iteration': self.max_iteration
        }
        self.crossing = 0
        self.mutate = 0
  
    def get_parameters(self):
        'Return default parmetrs in a dictionary'
        return self.hyperparameters
    

    def _find_the_best(self, population, score_list):
        'Choose an individual with the highest score'
        best_score = max(score_list)
        best_index = score_list.index(best_score)
        best_individual = population[best_index]
        return best_individual, best_score

    def _scaling(self, weights: list):
        if max(weights) != min(weights):
            return [(x - min(weights)/(max(weights) - min(weights))) for x in weights ]
        else:
            return [1 for x in weights]
            


    def _reproduction(self, population, score_list):
        'Roulette selection'
         
        best_ind, best_sc = self._find_the_best(population, score_list)
        population.remove(best_ind)
        score_list.remove(best_sc)
        size = len(population)

        weights = self._scaling(score_list)

        next_generation = random.choices(population, weights=weights, k=size)
        next_generation.append(best_ind)
        return next_generation


    def _corssing_individuals(self, indificual1, individual2):
        length = len(indificual1)

        crossing_point = random.randint(0, length)

        ind_2 = copy.deepcopy(individual2)

        for index in range (5, 9):
            individual2[index] = indificual1[index]
            indificual1[index] = ind_2[index]

        return indificual1, individual2
    

    def _choosing_if_we_exchange(self, possibility):
        'Draw betwen two possibilities taking in consideration probabilities of those elements'
        possibilities = [True, False]
        possibility_not = 1 - possibility
        weights = [possibility, possibility_not]
        
        decision = random.choices(possibilities, weights, k=1)

        return decision[0]

    
    def _crossing(self, population, probability):
        'Crossing individual in the given population'
        
        crossings_quantity = int(len(population)/2)
        population_quantity = len(population)
        indexes_to_cross = [x for x in range(population_quantity)]
        for i in range(crossings_quantity):
            rand_index1 = None
            rand_index2 = None
            while rand_index1 is None:
                rand_index1 = random.choice(indexes_to_cross)
            index_to_remove = indexes_to_cross.index(rand_index1)
            indexes_to_cross[index_to_remove] = None
            individual1 = copy.deepcopy(population[rand_index1])

            while rand_index2 is None:
                rand_index2 = random.choice(indexes_to_cross)
            index_to_remove = indexes_to_cross.index(rand_index2)
            indexes_to_cross[index_to_remove] = None
            individual2 = copy.deepcopy(population[rand_index2])
            
            if self._choosing_if_we_exchange(probability):
                self.crossing += 1
                individual1, individual2 = self._corssing_individuals(individual1, individual2)

                population[rand_index1] = individual1
                population[rand_index2] = individual2
                self.crossing += 1
        
        return population

    
    def _mutation(self, population: dict, probability):
        'Mutates genes in the given population'
        for individual in population:
            length = len(individual)
            for row in individual.values():
                for index in range(len(row)):
                    if self._choosing_if_we_exchange(probability):
                        self.mutate += 1
                        index_to_exchange = random.choice(range(len(row)))
                        row[index], row[index_to_exchange] = row[index_to_exchange], row[index]
                        # self.mutation +=1
                        
        return population

    def _update_parameters(self, kwargs):
        self.quantity_of_population = self.quantity_of_population if "quantity_of_population" not in kwargs else kwargs["quantity_of_population"]
        self.mutation_possibility = self.mutation_possibility if "mutation_possibility" not in kwargs else kwargs["mutation_possibility"]
        self.crossing_possibility = self.crosing_possibility if "crossing_possibility" not in kwargs else kwargs["crossing_possibility"]
        self.max_iteration = self.max_iteration if "max_iteration" not in kwargs else kwargs["max_iteration"]


    def solve(self, problem, pop0, *args, **kwargs):
        'Activates the genetic algorithm'
        # Initialization

        self._update_parameters(kwargs)

        quantity_of_population = self.quantity_of_population
        mutation_possibility = self.mutation_possibility
        crossing_possibility = self.crosing_possibility
        max_iteration = self.max_iteration
        
        population = pop0[0:quantity_of_population]

        score_list = [problem(individual) for individual in population] 

        best_individual, best_score = self._find_the_best(population, score_list)
    
        for i in range(max_iteration):
    
            population = self._reproduction(population, score_list)
           
            population = self._crossing(population, crossing_possibility)
         
            population = self._mutation(population, mutation_possibility)
            score_list = [problem(individual) for individual in population]
            mean = statistics.mean(score_list)
            print(f"Mean score in {i} iterations: {mean}")
            write_to_file(f"Mean score in {i} iterations: {mean}")
            best_individual_trial, best_score_trial = self._find_the_best(population, score_list)
            print(f"Best score in {i} iterations: {best_score_trial}")
            write_to_file(f"Best score in {i} iterations: {best_score_trial}")
      
            if best_score_trial > best_score:
                best_individual = best_individual_trial
                best_score = best_score_trial
            if best_score == 162:
                return best_individual, best_score
            
        return best_individual, best_score
    

def write_to_file(line):
    with open ('output.txt', 'a') as file:
        file.write(line + '\n')
        

        

In [12]:


values = [
    1, 9, 0, 0, 2, 0, 5, 0, 8,
    0, 6, 7, 0, 0, 0, 0, 4, 0,
    0, 0, 4, 6, 8, 3, 0, 9, 0,
    3, 0, 0, 7, 0, 0, 2, 0, 9,
    0, 0, 0, 1, 0, 0, 6, 0, 5,
    0, 0, 0, 5, 9, 8, 0, 0, 4,
    4, 0, 5, 8, 0, 0, 9, 0, 6,
    2, 0, 6, 0, 4, 0, 0, 5, 1,
    9, 0, 1, 0, 0, 6, 0, 7, 0
]     

sudoku = Sudoku(values)
population = sudoku.get_population(500)
gen_alg = GeneticAlgorithm()
sudoku.display_board(sudoku.rows)

best_individual, best_score = gen_alg.solve(sudoku.fitness_function, population, max_iteration=60000, crossing_possibility=0.2
                                            , mutation_possibility=0.001)
if best_score == 162:
    print("Find Solution")
    write_to_file("Find Solution")
else:
    print("Couldn't find solution")
    write_to_file("Couldn't find solution")
ind_rows = sudoku._get_individual_rows(best_individual)

print(f"The best score: {best_score}")
write_to_file(f"The best score: {best_score}")

print(f"The best individual: {best_individual}")
write_to_file(f"The best individual: {best_individual}")
sudoku.display_board(ind_rows)

1 9 * | * 2 * | 5 * 8 
* 6 7 | * * * | * 4 * 
* * 4 | 6 8 3 | * 9 * 
- - - - - - - - - - - 
3 * * | 7 * * | 2 * 9 
* * * | 1 * * | 6 * 5 
* * * | 5 9 8 | * * 4 
- - - - - - - - - - - 
4 * 5 | 8 * * | 9 * 6 
2 * 6 | * 4 * | * 5 1 
9 * 1 | * * 6 | * 7 * 
Mean score in 0 iterations: 118.898
Best score in 0 iterations: 134
Mean score in 1 iterations: 119.348
Best score in 1 iterations: 134
Mean score in 2 iterations: 119.518
Best score in 2 iterations: 135
Mean score in 3 iterations: 119.516
Best score in 3 iterations: 133
Mean score in 4 iterations: 119.838
Best score in 4 iterations: 136
Mean score in 5 iterations: 120.07
Best score in 5 iterations: 137
Mean score in 6 iterations: 119.978
Best score in 6 iterations: 136
Mean score in 7 iterations: 120.014
Best score in 7 iterations: 134
Mean score in 8 iterations: 119.812
Best score in 8 iterations: 136
Mean score in 9 iterations: 120.096
Best score in 9 iterations: 134
Mean score in 10 iterations: 120.044
Best score in 10 iterations: 13